In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Problem
- As a company with tight financial. The company wants to provide health-mental wellness program and health examination program for employees that TENDS to absent. But it is hard for the company to indentify which employees that needs to be treated in the program.

# Goals
- Identifying the employee that TENDS to absent
- find the factors based on the data that affecting the healthiness of employee
- the manager is ready enough to handle if their employee are tends go absent

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from dython.nominal import associations, cramers_v, correlation_ratio, theils_u


pd.options.display.max_columns = 999
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('/kaggle/input/absenteeism-at-work-uci-ml-repositiory/Absenteeism_at_work.csv', sep = ';')
df

# Data Cleaning

In [ ]:
bsDesc = []

for i in df.columns:
    bsDesc.append([
        i,
        df[i].dtypes,
        df[i].isna().sum(),
        round((((df[i].isna().sum() )/ len(df)) * 100),2),
        df[i].nunique(),
        df[i].drop_duplicates().sample(2).values
    ])
pd.DataFrame(data = bsDesc,
            columns = [
                'Features',
                'DataType',
                'Null',
                'NullPercentage',
                'Unique',
                'Unique Sample'
            ])

Data is clean

In [ ]:
df.describe()

In [ ]:
df.dtypes

In [ ]:
df.drop(columns = 'ID', inplace = True)

In [ ]:
outliers = []
def outlier_check(data):
    sorted(data)
    q1, q3 = np.percentile(data, [25,75])
    iqr = q3-q1
    lower_fence = q1 - (1.5 * iqr)
    upper_fence = q3 + (1.5 * iqr)
    for i in data:
        if i < lower_fence or i > upper_fence:
            outliers.append(i)
    return outliers

In [ ]:
outliers = []
len(outlier_check(df['Age']))

In [ ]:
df_outlierage = df[df['Age'].isin(outliers)]
df_outlierage

In [ ]:
plt.figure(figsize=(10,8))
sns.boxplot(data=df['Age'])
plt.title("Amount of outliers in Age")
plt.xticks(rotation=45)
plt.show()

- data is clean
- no missing value
- all rows are numeric values
- data target is absenteeism time in hours -> continous numeric -> we are going to change the target into 1 (absent) and 0 (not absent) later on for machine learning purposes.
- based on df.describe() data is well prepared, the min and max value is according to its description. And there are no outliers that we need to drop

# EDA

In [ ]:
plt.figure(figsize = (10,8))
sns.distplot(df['Age'],label = 'Skewness : %.2f'%(df['Age'].skew()))
plt.title('distribution of age data')
plt.legend()

plt.plot()
plt.show()

The distribution of age is moderately skewed with skewness score = 0.7. Also positive skew which means that the mode is  smaller than the median or mean. Based on the data the majority of employee's age is around 20 to 60. 

In [ ]:
mean_reason = df[['Reason for absence', 'Absenteeism time in hours']].groupby('Reason for absence').mean().reset_index().sort_values('Absenteeism time in hours', ascending = False)
plt.figure(figsize = (20,5))
sns.barplot(mean_reason['Reason for absence'],   mean_reason['Absenteeism time in hours'])
plt.title('mean absent hours based on reason of absence')
plt.show()

The highest mean hours of absensce is because of disease of circulatory system. It is related to heart disease, based on the data heart disease took longer time to recover, it is 18 hours longer than the top 2 absent reason which is neoplasm. 

In [ ]:
reason_counts = df[['Reason for absence', 'Absenteeism time in hours']].groupby('Reason for absence').count().reset_index().sort_values('Absenteeism time in hours', ascending = False)
reason_counts.columns = ['Reason for absence', 'Absenteeism count']
plt.figure(figsize = (20,5))
sns.barplot(reason_counts['Reason for absence'],   reason_counts['Absenteeism count'])
plt.title('absenteeism count based on reason for absence')
plt.show()

In [ ]:
reason_sum = df[['Reason for absence', 'Absenteeism time in hours']].groupby('Reason for absence').sum().reset_index().sort_values('Absenteeism time in hours', ascending = False)
reason_sum.columns = ['Reason for absence', 'Total numbers of hours']
plt.figure(figsize = (20,5))
sns.barplot(reason_sum['Reason for absence'],   reason_sum['Total numbers of hours'])
plt.title('total absent hours based on reason of absence')
plt.show()

top 3 total number of hours :
- Diseases of the musculoskeletal system and connective tissue  
  because of limitation of data (we dont know that type of this company is) so basecdon the disease we assume that lots of the company are manual labour (hard worker) that use its muscle to work. it leads to the highest hours of absent. 
- Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified  
- medical consultation
  medical consultation is common in reason of abscenses in company.

In [ ]:
bd_mass = df[['Reason for absence', 'Body mass index']].groupby('Reason for absence').mean().reset_index().sort_values('Body mass index', ascending = False)
plt.figure(figsize = (20,5))
sns.barplot(x = bd_mass['Reason for absence'], y = bd_mass['Body mass index'])
plt.show()

Underweight = <18.5
Normal weight = 18.5–24.9
Overweight = 25–29.9
Obesity = BMI of 30 or greater
- the mean body mass index in the company is 26.6 -> overweight


based on the data, obesity employees (bmi >= 30) has reason of absence :
- unjustified reason
- mental and behavioral disorder
- blood donation
- surprisingly, the person that donates blood came from obesity person. based on article ,there is no upper weight limit as long as your weight is not higher than the weight limit of the donor bed or lounge you are using.
- well unjustified reason needs to be describe more specificly to know that does obesity leads to serious problem.
- mental and behavioral disorder are from obese person -> company needs to provide mental wellness care
- surprisingly the person who are healthy (no absent) are from overweight class. As long as the person can maintain its healthy-life balance they has lower probability of sickness

In [ ]:
month = df[['Month of absence', 'Absenteeism time in hours']].groupby('Month of absence').sum().reset_index()
plt.figure(figsize = (10,5))
sns.lineplot(x = month['Month of absence'], y= month['Absenteeism time in hours'])
plt.show()

In march and july employee tends to go absent. Company should take a step to lower the number of absentees in this month. Higher hours of absentees can leads to lower productivity to the company. 

In [ ]:
day = df[['Day of the week', 'Absenteeism time in hours']].groupby('Day of the week').sum().reset_index()
day
plt.figure(figsize = (10,5))
sns.lineplot(x = day['Day of the week'], y= day['Absenteeism time in hours'])
plt.plot()

There were no data that recorded on monday (0 records).
- The more closer to weekend, the lower the total absenteeism hours

In [ ]:
season = df[['Seasons', 'Absenteeism time in hours']].groupby('Seasons').sum().reset_index()
plt.figure(figsize = (10,5))
sns.lineplot(x = season['Seasons'], y = season['Absenteeism time in hours'])
plt.show()

Despite having the highest total hours of absence in march, if we see from the graph above. Season 1 (Q1) from jaunary to march is not the highest total hours of absence.
The highest is from season s3(Q3) from july to september. 

In [ ]:
sns.scatterplot(df['Transportation expense'], df['Absenteeism time in hours'])
plt.show()

If we take a look at the scatter plot, the were no linear relationship between transporation expense and absenteeism time in hours

In [ ]:
sns.heatmap(df[['Transportation expense', 'Absenteeism time in hours']].corr('spearman'), annot = True)
plt.show()

Based on confusion matrix, there are positive correlation but the number is quite low (weak correlation).

In [ ]:
sns.scatterplot(df['Distance from Residence to Work'], df['Absenteeism time in hours'])
plt.show()

In [ ]:
sns.heatmap(df[['Distance from Residence to Work', 'Absenteeism time in hours']].corr('spearman'), annot = True)
plt.show()

- If we take a look at the scatter plot, the were no  linear relationship between distance and absenteeism time in hours. Based on confusion matrix, there are positive correlation but the number is indicating that it is very weak because the number is very close to 0. We could conclude that distance is not the factors that made the employee absent.

In [ ]:
sns.scatterplot(df['Service time'], df['Absenteeism time in hours'])
plt.show()

In [ ]:
sns.heatmap(df[['Service time', 'Absenteeism time in hours']].corr('spearman'), annot = True)
plt.show()

- If we take a look at the scatter plot, the were no  linear relationship between service time and absenteeism time in hours. Based on confusion matrix, there are negative correlation but the number is indicating that it is very weak because the number is very close to 0. We could conclude that service time  is not the factors that made the employee absent.

In [ ]:
sns.scatterplot(df['Age'], df['Absenteeism time in hours'])
plt.show()

In [ ]:
sns.heatmap(df[['Age', 'Absenteeism time in hours']].corr('spearman'), annot = True)
plt.show()

- If we take a look at the scatter plot, the were no  linear relationship between age and absenteeism time in hours. Based on confusion matrix, there are negative correlation but the number is indicating that it is very weak because the number is very close to 0. We could conclude that age is not the factors that made the employee absent. Employee that is on its senior age does not mean that they are more prone to sickness, also employee that are still young does not mean they're healthy.

In [ ]:
sns.scatterplot(df['Work load Average/day '], df['Absenteeism time in hours'])
plt.show()

In [ ]:
sns.heatmap(df[['Work load Average/day ', 'Absenteeism time in hours']].corr('spearman'), annot = True)
plt.show()

- If we take a look at the scatter plot, the were no  linear relationship between workload and absenteeism time in hours. Based on confusion matrix, there are positive correlation but the number is indicating that it is very weak because the number is very close to 0. We could conclude that workload is not the factors that made the employee absent. Usually higher workload leads to more stress for the employee, but based on this data higher workload does not affecting their mental health. 

In [ ]:
sns.scatterplot(df['Hit target'], df['Absenteeism time in hours'])
plt.show()

In [ ]:
sns.heatmap(df[['Hit target', 'Absenteeism time in hours']].corr('spearman'), annot = True)
plt.show()

- If we take a look at the scatter plot, the were no  linear relationship between Hit target and absenteeism time in hours. Based on confusion matrix, there are positive correlation but the number is indicating that it is very weak because the number is very close to 0. We could conclude that hit target is not the factors that made the employee absent.

In [ ]:
df[['Disciplinary failure', 'Absenteeism time in hours']].groupby('Disciplinary failure').sum()

- The employee that are absent does not because of indisciplinary action
- the indisiciplinary action is not because of they are absent with no reason. but it is more indiciplinary action in work environment

In [ ]:
df[['Education', 'Absenteeism time in hours']].groupby('Education').count()

In [ ]:
edu= df[['Education', 'Absenteeism time in hours']].groupby('Education').sum().reset_index()
sns.barplot(edu['Education'], edu['Absenteeism time in hours'])
plt.plot()

In [ ]:
correlation_ratio(df['Education'], df['Absenteeism time in hours'])

- the majority education of employee is on highschool level and the majority of absenteeism are from high-school education - employee.
- based on correlation ratio, higher education does not mean they are healthier.

In [ ]:
son= df[['Son', 'Absenteeism time in hours']].groupby('Son').sum().reset_index()
sns.barplot(son['Son'], son['Absenteeism time in hours'])
plt.plot()

In [ ]:
df[['Son', 'Absenteeism time in hours']].corr('spearman')

- surprisingly, the more number of son they have the less hours they are absence
- based on correlation ratio, more son does not affecting absence hours.

In [ ]:
drink= df[['Social drinker', 'Absenteeism time in hours']].groupby('Social drinker').sum().reset_index()
sns.barplot(drink['Social drinker'], son['Absenteeism time in hours'])
plt.show()

- Social drinker category does not leads to more absenteism time in hours

In [ ]:
smoke= df[['Social smoker', 'Absenteeism time in hours']].groupby('Social smoker').sum().reset_index()
sns.barplot(smoke['Social smoker'], son['Absenteeism time in hours'])
plt.plot()

- surprisingly smoking does not leads to more absenteeism
- based on correlation ratio, smoking does not affecting the hours of absenteeism

In [ ]:
pet= df[['Pet', 'Absenteeism time in hours']].groupby('Pet').sum().reset_index()
sns.barplot(pet['Pet'], pet['Absenteeism time in hours'])
plt.show()

- based on the data, employee that have pet are having less in absenteeism time in hours. Having a pet its one of source of hapiness for some person, it could boost immunity if the person is happy.

In [ ]:
bmi= df[['Body mass index', 'Absenteeism time in hours']].groupby('Body mass index').sum().reset_index()
sns.barplot(bmi['Body mass index'], bmi['Absenteeism time in hours'])
plt.show()

In [ ]:
df[['Body mass index', 'Absenteeism time in hours']].corr('spearman')

- bmi does not affecting to absenteesim time in hours based on the barplot and correlation (-0.06)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, recall_score, accuracy_score, f1_score, confusion_matrix, precision_score
from sklearn.svm import SVC

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler, PowerTransformer, PolynomialFeatures, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from xgboost import XGBClassifier, XGBRegressor

In [ ]:
b = []
for i in df['Absenteeism time in hours']:
    if i > 0 :
        b.append(1)
    else:
        b.append(0)
df['Absent'] = b #classification target

In [ ]:
df.drop(columns = ['Reason for absence', 'Absenteeism time in hours'], inplace = True) #it will be obvious if we include the reason for absence and absenteeism time
df.drop(columns = ['Month of absence', 'Day of the week'], inplace = True) #it will be obvious if we include the month of absence and day of week. We use seasons instead (season more inmpactful to person's health)


In [ ]:
X = df.drop(columns = 'Absent')
y = df['Absent']
X_train, X_test, y_train, y_test = train_test_split(X,y, stratify = y , test_size = 0.3, random_state =42)

In [ ]:
df['Absent'].value_counts(normalize = True)

Minority = not absent
majority = absent
1 = absent (not present)
0 = not absent (present)
True positive = Actual absent, prediction absent
True negative = Actual not absent, prediction not absent
False positive =  Actual not absent (present), but the prediction is absent
False negative = Actual absent but the prediction is present -> main concern 

MAIN CONCERN
- our main concern is to supress the number of false negative while also having least number of false positive. (balancing)
- Why we want to balance out the false positive and false negative. 
  Case:
  After detecting the employee that tends to go sick. the company apporach is to examine the employees health to mental-wellness-program and healthy-program. Also taking several health examination. If we have big false positive then company have to pay more for the program and examination (more people that detected tends to go absent but the actual he is healthy). Also we need to supress the false negative so that our prediction is more likely to be accurate in predicting people that tends to go absent.
  
Because the data is imbalance so we focus on tuning the minority class which is 0 (not absent) to get more accurate prediction

In [ ]:
# Base model
LR = LogisticRegression()
LR.fit(X_train,y_train)
y_LR_test = LR.predict(X_test)
print(classification_report(y_test, y_LR_test))
cm_LR_test = confusion_matrix(y_test, y_LR_test, labels = [1,0])
df_LR_test = pd.DataFrame(data=cm_LR_test, columns=['Pred 1', 'Pred 0'], index=['Akt 1', 'Akt 0'])
sns.heatmap(df_LR_test, annot=True)
plt.show()

In [ ]:

KNN = KNeighborsClassifier()
KNN.fit(X_train,y_train)
y_KNN_test = KNN.predict(X_test)
print(classification_report(y_test, y_KNN_test))
cm_KNN_test = confusion_matrix(y_test, y_KNN_test, labels = [1,0])
df_KNN_test = pd.DataFrame(data=cm_KNN_test, columns=['Pred 1', 'Pred 0'], index=['Akt 1', 'Akt 0'])
sns.heatmap(df_KNN_test, annot=True)
plt.show()

In [ ]:
SVM = SVC()
SVM.fit(X_train,y_train)
y_SVM_test = SVM.predict(X_test)
print(classification_report(y_test, y_SVM_test))
cm_SVM_test = confusion_matrix(y_test, y_SVM_test, labels = [1,0])
df_SVM_test = pd.DataFrame(data=cm_SVM_test, columns=['Pred 1', 'Pred 0'], index=['Akt 1', 'Akt 0'])
sns.heatmap(df_SVM_test, annot=True)
plt.show()


In [ ]:
DT = DecisionTreeClassifier()
DT.fit(X_train,y_train)
y_DT_test = DT.predict(X_test)
print(classification_report(y_test, y_DT_test))
cm_DT_test = confusion_matrix(y_test, y_DT_test, labels = [1,0])
df_DT_test = pd.DataFrame(data=cm_KNN_test, columns=['Pred 1', 'Pred 0'], index=['Akt 1', 'Akt 0'])
sns.heatmap(df_DT_test, annot=True)
plt.show()

In [ ]:
RF = RandomForestClassifier()
RF.fit(X_train,y_train)
y_RF_test = RF.predict(X_test)
print(classification_report(y_test, y_RF_test))
cm_RF_test = confusion_matrix(y_test, y_RF_test, labels = [1,0])
df_RF_test = pd.DataFrame(data=cm_RF_test, columns=['Pred 1', 'Pred 0'], index=['Akt 1', 'Akt 0'])
sns.heatmap(df_RF_test, annot=True)
plt.show()

# Conclusion on base model
- Random forest has the best recall  in the MINORITY CLASS.
- Decision Tree 2nd best but it has higher false positive
- KNN and SVM recall-precision score is far from accurate
- Logreg still can be improved

In [ ]:
pipeline_LR = Pipeline([
    ('scale', StandardScaler()),
    ('algo', LogisticRegression())
])
pipeline_KNN = Pipeline([
    ('scale', StandardScaler()),
    ('algo', KNeighborsClassifier())
])
pipeline_SVM = Pipeline([
    ('scale', StandardScaler()),
    ('algo', SVC())
])
pipeline_RF = Pipeline([
    ('scale', StandardScaler()),
    ('algo', RandomForestClassifier())
])


In [ ]:
pipeline_LR.fit(X_train, y_train)
y_LRT_test = pipeline_LR.predict(X_test)
print(classification_report(y_test, y_LRT_test))
cm_LRT_test = confusion_matrix(y_test, y_LRT_test, labels = [1,0])
df_LRT_test = pd.DataFrame(data=cm_LRT_test, columns=['Pred 1', 'Pred 0'], index=['Akt 1', 'Akt 0'])
sns.heatmap(df_LRT_test, annot=True)
plt.show()


In [ ]:
pipeline_KNN.fit(X_train, y_train)
y_KNNT_test = pipeline_KNN.predict(X_test)
print(classification_report(y_test, y_KNNT_test))
cm_KNNT_test = confusion_matrix(y_test, y_KNNT_test, labels = [1,0])
df_KNNT_test = pd.DataFrame(data=cm_KNNT_test, columns=['Pred 1', 'Pred 0'], index=['Akt 1', 'Akt 0'])
sns.heatmap(df_KNNT_test, annot=True)
plt.show()

In [ ]:
pipeline_SVM.fit(X_train, y_train)
y_SVM_test = pipeline_SVM.predict(X_test)
print(classification_report(y_test, y_SVM_test))
cm_SVM_test = confusion_matrix(y_test, y_SVM_test, labels = [1,0])
df_SVM_test = pd.DataFrame(data=cm_SVM_test, columns=['Pred 1', 'Pred 0'], index=['Akt 1', 'Akt 0'])
sns.heatmap(df_SVM_test, annot=True)
plt.show()

In [ ]:
pipeline_RF.fit(X_train, y_train)
y_RF2_test = pipeline_RF.predict(X_test)
print(classification_report(y_test, y_RF2_test))
cm_RF2_test = confusion_matrix(y_test, y_RF2_test, labels = [1,0])
df_RF2_test = pd.DataFrame(data=cm_RF2_test, columns=['Pred 1', 'Pred 0'], index=['Akt 1', 'Akt 0'])
sns.heatmap(df_RF2_test, annot=True)
plt.show()


# conclusion using scalling
- well all of the algoritm seems to have great score after scalling. recall = 0.85

# Hyperparameter Tuning

In [ ]:
param_KNN = {
    'algo__n_neighbors': np.arange(1,51,2),
    "algo__p" : [1,2],
    'algo__weights' : ['uniform', 'distance']
}
skf = StratifiedKFold(n_splits = 3)
KNN_GS = GridSearchCV(pipeline_KNN, param_KNN, cv = skf, scoring = 'f1', n_jobs= -1, verbose = 1)
KNN_GS.fit(X_train,y_train)

In [ ]:
KNN_GS.best_params_

In [ ]:
KNN_Tuned = KNN_GS.best_estimator_
y_KNN2_test = KNN_Tuned.predict(X_test)
print(classification_report(y_test, y_KNN2_test))
cm_KNN2_test = confusion_matrix(y_test, y_KNN2_test, labels = [1,0])
df_KNN2_test = pd.DataFrame(data=cm_KNN2_test, columns=['Pred 1', 'Pred 0'], index=['Akt 1', 'Akt 0'])
sns.heatmap(df_SVM_test, annot=True)
plt.show()

In [ ]:
param_SVM = {
    'algo__C' : np.logspace(-3,3,7),
    'algo__gamma' : np.arange(10, 101, 10),
    'algo__class_weight': [{0 : 1- x, 1 : x} for x in [.1, .2, .3, .4, .5]]
}
SVM_GS = GridSearchCV(pipeline_SVM, param_SVM, cv = skf, scoring = 'recall', n_jobs= -1, verbose = 1)
SVM_GS.fit(X_train,y_train)

In [ ]:
SVM_Tuned = SVM_GS.best_estimator_
y_SVMT_test = SVM_Tuned.predict(X_test)
print(classification_report(y_test, y_SVMT_test))
cm_SVMT_test = confusion_matrix(y_test, y_SVMT_test, labels = [1,0])
df_SVMT_test = pd.DataFrame(data=cm_SVMT_test, columns=['Pred 1', 'Pred 0'], index=['Akt 1', 'Akt 0'])
sns.heatmap(df_SVMT_test, annot=True)
plt.show()

In [ ]:
param_RF = {
    "n_estimators" : np.arange(100, 501, 100), 
    "max_depth" : [None, 5, 10, 15], 
    "min_samples_leaf" : np.arange(1, 17, 5), 
    "max_features" : [0.3, 0.5, 0.7, 0.8],
    "class_weight" : [{0 : 1- x, 1 : x} for x in [.1, .2, .3, .4, .5]]
}
RF_T = GridSearchCV(RandomForestClassifier(), param_RF, cv=skf, n_jobs=-1, verbose=1, scoring='recall')
RF_T.fit(X_train, y_train)

In [ ]:
RF_Tuned = RF_T.best_estimator_
y_RF3_test = RF_Tuned.predict(X_test)
print(classification_report(y_test, y_RF3_test))
cm_RF3_test = confusion_matrix(y_test, y_RF3_test, labels = [1,0])
df_RF3_test = pd.DataFrame(data=cm_RF3_test, columns=['Pred 1', 'Pred 0'], index=['Akt 1', 'Akt 0'])
sns.heatmap(df_RF3_test, annot=True)
plt.show()

It seems like the recall score still 0.85 (our best recall score), so we use another metrics called ROC AUC. AUC - ROC curve is a performance measurement for the classification problems at various threshold settings. ROC is a probability curve and AUC represents the degree or measure of separability. It tells how much the model is capable of distinguishing between classes. Higher the AUC, the better the model is at predicting 0s as 0s and 1s as 1s. By analogy, the Higher the AUC, the better the model is at distinguishing between patients with the disease and no disease.

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc

In [ ]:
y_proba = pipeline_LR.predict_proba(X_test)
pred_1 = y_proba[:, 1]
roc_curve(y_test, pred_1)

In [ ]:
fpr, tpr, threshold = roc_curve(y_test, pred_1)
# AUC Score = Area Under Curve
auc_score = round((auc(fpr, tpr)), 2)
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, 'b', label = f"AUC = {auc_score}")
plt.plot([0, 1], [0, 1], 'r-.')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic Curve of Logistic regression after feature engineering')
plt.legend()
plt.show()

In [ ]:
y_proba = KNN_Tuned.predict_proba(X_test)
pred_1 = y_proba[:, 1]
roc_curve(y_test, pred_1)
fpr, tpr, threshold = roc_curve(y_test, pred_1)
auc_score = round((auc(fpr, tpr)), 2)
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, 'b', label = f"AUC = {auc_score}")
plt.plot([0, 1], [0, 1], 'r-.')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic Curve of KNN hyperparameter tuning')
plt.legend()
plt.show()

In [ ]:
y_proba = DT.predict_proba(X_test)
pred_1 = y_proba[:, 1]
roc_curve(y_test, pred_1)
fpr, tpr, threshold = roc_curve(y_test, pred_1)
auc_score = round((auc(fpr, tpr)), 2)
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, 'b', label = f"AUC = {auc_score}")
plt.plot([0, 1], [0, 1], 'r-.')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic Curve of DecisionTree ')
plt.legend()
plt.show()

In [ ]:
y_proba = RF_Tuned.predict_proba(X_test)
pred_1 = y_proba[:, 1]
roc_curve(y_test, pred_1)
fpr, tpr, threshold = roc_curve(y_test, pred_1)
auc_score = round((auc(fpr, tpr)), 2)
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, 'b', label = f"AUC = {auc_score}")
plt.plot([0, 1], [0, 1], 'r-.')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic Curve of random forest after Hyperparameter tuning')
plt.legend()
plt.show()

# Conclusion
- the best algoritm for this classification problem is using randomforest algoritm with area under the curve (AUC = 0.95) and recall score 0.85
- the recall score random forest before tuned and after tuned is the same also the AUC score
- tuning does not improve the model recall score

# EDA Conclusion
- bmi does not affecting to absenteesim time in hours based on the barplot and correlation (-0.06)
- based on the data, employee that have pet are having less in absenteeism time in hours. Having a pet its one of source of hapiness for some person, it could boost immunity if the person is happy.
- surprisingly smoking does not leads to more absenteeism 
- Social drinker category leads to more absenteism time in hours
- surprisingly, the more number of son they have the less hours they are absence. based on correlation ratio, more son does not affecting absence hours.
- the majority education of employee is on highschool level and the majority of absenteeism are from high-school education - employee. based on correlation ratio, higher education does not mean they are healthier.
- The employee that are absent does not because of indisciplinary action
- the indisiciplinary action is not because of they are absent with no reason. but it is more indiciplinary action in work environment
-  We could conclude that hit target is not the factors that made the employee absent.
-  We could conclude that workload is not the factors that made the employee absent. Usually higher workload leads to more stress for the employee, but based on this data higher workload does not affecting their mental health. 
- We could conclude that age is not the factors that made the employee absent. Employee that is on its senior age does not mean that they are more prone to sickness, also employee that are still young does not mean they're healthy.
- We could conclude that service time  is not the factors that made the employee absent
- We could conclude that distance is not the factors that made the employee absent.
- If we take a look at the scatter plot, the were no linear relationship between transporation expense and absenteeism time in hours, weak correlation
- Despite having the highest total hours of absence in march, if we see from the graph above. Season 1 (Q1) from jaunary to march is not the highest total hours of absence.The highest is from season s3(Q3) from july to september.
- The more closer to weekend, the lower the total absenteeism hours
- In march and july employee tends to go absence. Company should take a step to lower the number of absentees in this month. Higher hours of absentees can leads to lower productivity to the company. 
- The highest mean hours of absensce is because of disease of circulatory system. It is related to heart disease, based on the data heart disease took longer time to recover, it is 18 hours longer than the top 2 absent reason which is neoplasm 
- blood donation has the highest number of employee that absent, followed by dental consultation, physioteraphy, and musculoskeletal system. Well the employee is really care about humanity lots of them donates blood. Blood donation average hour absence is 2.8 hours per employee it is relatively low (ranked 4 from bottom of mean absence hours) it does not contribute much to the absent hours of employee per employee. if the duration of the absence is short, the work can still be done without disturbing the timeline of the project. But if the duration is high then the employee needs to find someone as a backup or the backup person might do 2 task in the same time which is leading to more stressed to the backup employee. So health is really a concern to maintain productivity of employee.
- Unjustified reason needs to be more specific because the number mean hours of absentees is 6.3 (higher than the median).

based on the data, obesity employees (bmi >= 30) has reason of absence :
- unjustified reason
- mental and behavioral disorder
- blood donation
- surprisingly, the person that donates blood came from obesity person. based on article ,there is no upper weight limit as long as your weight is not higher than the weight limit of the donor bed or lounge you are using.
- well unjustified reason needs to be describe more specificly to know that does obesity leads to serious problem.
- mental and behavioral disorder are from obese person -> company needs to provide mental wellness care
- surprisingly the person who are healthy (no absent) are from overweight class. As long as the person can maintain its healthy-life balance they has lower probability of sickness

# Machine learning conclusion and recommendation
Minority = not absent
majority = absent
1 = absent (not present)
0 = not absent (present)
True positive = Actual absent, prediction absent
True negative = Actual not absent, prediction not absent
False positive =  Aktualnya tidak absen (masuk), tapi diprediksi absen (tidak masuk)
False negative = Aktual nya dia absen (tidak masuk) tapi prediksi nya ga absen (masuk) -> main concern 

MAIN CONCERN
- our main concern is to supress the number of false negative while also having least number of false positive. (balancing)
- Why we want to balance out the false positive and false negative. 
  Case:
  After detecting the employee that tends to go sick. the company apporach is to examine the employees health to mental-wellness-program and healthy-program. Also taking several health examination. If we have big false positive then company have to pay more for the program and examination (more people that detected tends to go absent but the actual he is healthy). Also we need to supress the false negative so that our prediction is more likely to be accurate in predicting people that tends to go absent.
  
Because the data is imbalance so we focus on tuning the minority class which is 0 (not absent) to get more accurate prediction


- The recall score for base model is already high, so we do not need to do imbalance data handling
- the best algoritm for this classification problem is using randomforest algoritm with area under the curve (AUC = 0.95) and recall score 0.85
- the recall score random forest before tuned and after tuned is the same also the AUC score
- tuning does not improve the model recall score
I recommend the company to gather more health data about the employee such as (glucose level, cholestrol, uric acid, blood pressure and so on) and the data needs to be updated every terms. So that it gave more accurate readings of the prediction. Healthy employee leads to increase in productivity and low absenteeism.

Handling people that tends to go absent 
- mental wellness program
- health care program
- health examination

Assumption
- cost of medical program per employee = 10 USD
- if we provide medical program for all employee : 740 * 10 USD = 7400 USD
- if we use machine learning as the predictor, the company just pay based on the prediction (not paying for all employee)
- For example : using machine learning it predicts that there are 120 people tends to go absent.
    120 x 10 USd = 1200 USD
    7400 -1200  = 6200 USD -> SAVED.

- in longrun it will increase the productivity